In [3]:
import json, os, zipfile
import numpy as np

# defaults
seed = 1234
num_chains = 50
num_samples = 10000
num_dim = 10
model_name = "funnel10"

In [4]:
rng = np.random.default_rng(seed)

def generate_funnel(funnel_dim): # [chain_num, n_samples, params_dim]
    # generate parameters
    y_raw = rng.normal(loc=0, scale=1, size=(num_chains, num_samples, 1))
    
    x_raw = rng.normal(loc=0, scale=1, size=(num_chains, num_samples, funnel_dim - 1))
    
    # transform and return parameters
    y = 3 * y_raw
    x = np.exp(y_raw / 2) * x_raw
    return np.concatenate((y, x), axis=2)

ref_draws_np = generate_funnel(num_dim)   # [chain_num, n_samples, params_dim]

In [5]:
# convert from numpy 3d array to json format

param_names = [
    "v", "theta[1]", "theta[2]", "theta[3]", "theta[4]", "theta[5]", 
    "theta[6]", "theta[7]", "theta[8]", "theta[9]"
]

ref_draws = []
for chain in ref_draws_np:  # each chain is [n_samples, params_dim]
    chain_dict = {}
    for name, params in zip(param_names, chain.T):
        chain_dict[name] = params.tolist()
    ref_draws.append(chain_dict)

# save to file
with zipfile.ZipFile(f"{model_name}.ref_draws.json.zip", mode="w", compression=zipfile.ZIP_DEFLATED, compresslevel=9) as zip_file: 
    dumped_JSON: str = json.dumps(ref_draws, ensure_ascii=False, indent=4)
    zip_file.writestr(f"{model_name}.ref_draws.json", data=dumped_JSON)
    zip_file.testzip() # Test integrity of compressed archive
    
# with open(f"{model_name}.ref_draws.json", "w") as f:
#     json.dump(ref_draws, f)